In [42]:
import networkx as nx
import pandas as pd
import numpy as np
import eegraph
import stellargraph as sg
from spektral.data import Graph
import os
import pickle
from spektral.data import Dataset, DisjointLoader
import spektral 
import tensorflow as tf
from tensorflow.keras.optimizers import Adam


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool
from spektral.transforms import GCNFilter
from spektral.data import BatchLoader
from spektral.transforms.normalize_adj import NormalizeAdj
from spektral.layers.pooling import TopKPool
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy

from spektral.models.gcn import GCN


# Preprocesamiento

## Variables a usar

In [43]:
path = '../Scripts/data/data_vigilia'              #<--------------- PATH TO FOLDER CONTAINING EEGs
window_size_class_0 = 1                                 #<--------------- CLASS 0 WINDOW SIZE
window_size_class_1 = 1                               #<--------------- CLASS 1 WINDOW SIZE
connectivity_number_total = 1                                
exclude = [] 
#CONNECTIVITY MEASURES USED
connectivity= "squared_coherence"
bands = ['theta', 'alpha', 'beta']

In [44]:
class MyDataset(Dataset):
    """
    A dataset of five random graphs.
    """
    def __init__(self, nodes, feats, **kwargs):
        self.graphs = nodes
        self.labels = feats

        super().__init__(**kwargs)


    def read(self):
        output=[]
        for i in range(len(self.graphs)):
            for j in range(len(self.graphs[i][0])):
                # por cada grupo de grafos creados accedemos a cada posicion del diccionario y le añadimos el label correspondiente
                output.append(
                    Graph(a=nx.adjacency_matrix(self.graphs[i][0][j]), y=labels[i][j])
                )

        return output

## funcion modelar Grafo

In [45]:
def modelate_with_different_connectivity(window_size,entry, G, connectivity_number_total, connectivity, bands):
    total_graphs, total_labels = [], []
    #recorremos el numero de conectividades elegidas
    for i in range(connectivity_number_total):
        #creamos nuestro grafo modelado y guardamos nuestra matriz de adyacencia(no nos hace falta pero son datos que podrían venir bien en un futuro)
        graphs, c_m = G.modelate(window_size = window_size, connectivity = connectivity, bands = bands) #<--------------- THRESHOLD , threshold = 0.2          
      
        for i in range(len(graphs)):
            total_labels.append(str(entry))
        total_graphs.append(graphs) 
      
    return  total_graphs, total_labels

## Abrir directorios e importar datos

In [46]:
def open_data_directories(path, window_size_class_0, window_size_class_1, exclude, connectivity_number_total,connectivity,bands):
    array_grafos= []
    array_labels=[]
    class_files = os.listdir(path)
    #recorremos todas las carpetas y ficheros del path dado
    for entry in class_files:
        eeg_files = os.listdir(path + '/' + entry)
        for eeg in eeg_files:
            eeg_path = (path + '/' + entry + '/' + eeg)
            print(eeg_path, entry)#imprimimos el path del archivo completo
            G = eegraph.Graph() #creamos un objeto de tipo eegraph
            G.load_data(path= eeg_path, exclude = exclude) # llamamos al objeto y cargamos el archivo seleccionado
            
            if(entry == '1'):
                window_size = window_size_class_1
            elif (entry == '0'):
                window_size = window_size_class_0
            #llamamos recursivamente a la función para modelar nuestro grafo con las medidas deseadas
            final_graphs, final_labels =  modelate_with_different_connectivity(window_size ,entry, G, connectivity_number_total,connectivity,bands)
            array_labels.append(final_labels)
            array_grafos.append(final_graphs)
           # labels = labels + final_labels
    return array_grafos,array_labels

## llamada a las funciones y guardado de datos

In [47]:
#configs
learning_rate = 1e-2  # Learning rate
epochs = 400  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 32  # Batch size

In [48]:
graphs,labels = open_data_directories(path, window_size_class_0, window_size_class_1, exclude, connectivity_number_total,connectivity,bands)


../Scripts/data/data_vigilia/0/1_vigilia con 1.edf 0
Extracting EDF parameters from d:\CEIEC\EEG-Epilepsy\EEG-Epilepsy\GCNN\Scripts\data\data_vigilia\0\1_vigilia con 1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

EEG Information.
Number of Channels: 25
Sample rate: 512.0 Hz.
Duration: 4.998 seconds.
Channel Names: ['EEG Fp1', 'EEG Fp2', 'EEG F3', 'EEG F4', 'EEG C3', 'EEG C4', 'EEG P3', 'EEG P4', 'EEG O1', 'EEG O2', 'EEG F7', 'EEG F8', 'EEG T3', 'EEG T4', 'EEG T5', 'EEG T6', 'EEG Fz', 'EEG Cz', 'EEG Pz', 'EEG FT9', 'EEG FT10', 'EEG T9', 'EEG T10', 'EEG TP9', 'EEG TP10']
Model Data.
Squared_coherence_Estimator()
Frequency Bands: ['delta', 'theta', 'alpha', 'beta', 'gamma'] [False, True, True, True, False]
Intervals:  [(0, 512.0), (512.0, 1024.0), (1024.0, 1536.0), (1536.0, 2048.0), (2048.0, 2559.0)]

Threshold: 0.65
Number of graphs created: 15
../Scripts/data/data_vigilia/0/1_vigilia con 10.edf 0
Extracting EDF parameters from d:\CEIEC\EEG-E

In [49]:
df=MyDataset(graphs,labels, transforms=NormalizeAdj())

## division del dataset en varios para la red 


In [50]:
idxs = np.random.permutation(len(df))
split_va, split_te = int(0.8 * len(df)), int(0.9 * len(df))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
data_tr = df[idx_tr]
data_va = df[idx_va]
data_te = df[idx_te]

## Utilizacion de un batch size y un loader de la libreria spektral

In [51]:
loader_tr = DisjointLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = DisjointLoader(data_va, batch_size=batch_size)
loader_te = DisjointLoader(data_te, batch_size=batch_size)

# Creacion de la GNN


In [63]:
class MyFirstGNN(Model):
     def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'softmax')

     def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

optimizer = Adam(lr=learning_rate)
loss_fn = CategoricalCrossentropy()

D:\Anaconda\envs\Ceiec\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


## aplicamos filtros para normalizar segun el paper 

In [64]:
model = MyFirstGNN(32, df.n_labels)
model.compile(optimizer, loss_fn)

### Hasta este punto, el código es funcional, a partir de aquí el problema radica en la propia red. El código es una implementación de cómo lo hacen varios ejemplos de la web oficial, combianando tanto la importación de datos y la conversion de objetos spektral hasta la implementación de la GCN, la cual es el modelo base que propone Kipf en su paper 

In [69]:

model.fit(loader_tr,steps_per_epoch=1000, epochs=10)

ValueError: `y` argument is not supported when using python generator as input.